# <B> Setup for SageMaker Immersion Day with DL Framework </B>
* Container: conda_python3

## AutoReload

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Set roles

In [26]:
from sagemaker import get_execution_role

In [27]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


### 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](images/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](images/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](images/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](images/attach-policy.png) 

In [28]:
from utils.iam import iam_handler

In [29]:
iam = iam_handler()

In [30]:
strID = "DJ"
strPrefix = f'{strID}-SM-IMD'

In [54]:
strPrefix

'DJ-SM-IMD'

### 1.2 Attach policy - sagemaker execution role
> SSM  
> S3 <BR>
> CodeCommit <BR>
> CodeBuild <BR>
> CodePipeline <BR>
> EventBridge <BR>
> Lambda

In [31]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess",
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess"
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

### 1.3 Create role for lambda

In [32]:
listService = ["lambda.amazonaws.com"]
strRoleName = "-".join([strPrefix, "LabmdaRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
]
strDescription="Role for Lambda to call SageMaker functions'"

In [33]:
lambdaRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-LabmdaRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'ef9bc0a2-2bbd-46bf-adb5-2a2e91b1ff4b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ef9bc0a2-2bbd-46bf-adb5-2a2e91b1ff4b', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 08 May 2023 08:08:17 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole
{'ResponseMetadata': {'RequestId': 'c00ffda3-4b99-437b-96dd-9a84fdb82ad2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c00ffda3-4b99-437b-96dd-9a84fdb82ad2', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 08 May 2023 08:08:17 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonSageMakerFullAccess
{'ResponseMetadata': {'RequestId': '8ccb1c7d-e8b5-4823-b10a-aa25a9191f6c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8ccb1c7d-e8b5-4823-b10a-aa25a9191f6c', 'content-type': 

### 1.4 role-codebuild

In [34]:
listService = ["codebuild.amazonaws.com", "sagemaker.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodeBuildRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/CloudWatchFullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
]
strDescription="Role for CodeBuild"

In [35]:
buildRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
print (f"CODE-BUILD-ROLE-ARN: {buildRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-CodeBuildRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'a70c6712-8b42-4800-89dd-15d3fe18560c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a70c6712-8b42-4800-89dd-15d3fe18560c', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 08 May 2023 08:08:18 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonSSMFullAccess
{'ResponseMetadata': {'RequestId': 'df128b89-ce9c-4c6a-b148-de44c75f0ea4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'df128b89-ce9c-4c6a-b148-de44c75f0ea4', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 08 May 2023 08:08:18 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess
{'ResponseMetadata': {'RequestId': '44d85618-55b6-4b6f-8222-87fa420f3479', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '44d85618-55b6-4b6f-8222-87fa420f3479', 'content-type': 'text/xml', 'content

### 1.5 role-codepipeline

In [36]:
listService = ["codepipeline.amazonaws.com", "codebuild.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodePipelineRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodeDeployFullAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess"
]
strDescription="Role for CodePipeline"

In [37]:
pipelineRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {pipelineRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-CodePipelineRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'a63b87d8-c812-471a-9bb1-109b0d1264f1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a63b87d8-c812-471a-9bb1-109b0d1264f1', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 08 May 2023 08:08:19 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess
{'ResponseMetadata': {'RequestId': 'f4926377-d895-4634-8d77-6658b1c2ad39', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f4926377-d895-4634-8d77-6658b1c2ad39', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 08 May 2023 08:08:19 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeCommitFullAccess
{'ResponseMetadata': {'RequestId': '8a7b913e-8874-402e-97b2-96f83b258552', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8a7b913e-8874-402e-97b2-96f83b258552', 'content-type': 'text/xml', '

### 1.6 role-eventbridge

In [38]:
listService = ["events.amazonaws.com"]
strRoleName = "-".join([strPrefix, "EventBridgeRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
]
strDescription="Role for EventBridge"

In [17]:
eventBridgeRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {eventBridgeRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-EventBridgeRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': '4fa5d938-8289-4e79-83c6-6ce01d2a7602', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4fa5d938-8289-4e79-83c6-6ce01d2a7602', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 08 May 2023 08:07:37 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess
  Service name: ['events.amazonaws.com'], 
  Role name: DJ-SM-IMD-EventBridgeRole, 
  Policys: ['arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'], 
  Role ARN: arn:aws:iam::419974056037:role/DJ-SM-IMD-EventBridgeRole
== COMPLETED ==
CODE-PIPELINE-ROLE-ARN: arn:aws:iam::419974056037:role/DJ-SM-IMD-EventBridgeRole


## 2. Set datasets and utils

In [44]:
import os
import boto3
import sagemaker
from utils.s3 import s3_handler
from utils.ssm import parameter_store

In [45]:
strRegionName=boto3.Session().region_name

s3 = s3_handler()
pm = parameter_store(strRegionName)

This is a S3 handler with [None] region.


### 2.1 download and move data

In [46]:
strTmpDir = "./dataset"
strInputDir = "cifar-10-batches-py"
os.makedirs(strTmpDir, exist_ok=True)

strFileName = os.path.join(strTmpDir, "cifar-10-python.tar.gz")
!wget -O $strFileName http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar xfvz $strFileName -C $strTmpDir

strDataPath, strTmpPath = os.path.join(strTmpDir, "data"), os.path.join(strTmpDir, strInputDir)
os.makedirs(strDataPath, exist_ok=True)
!mv $strTmpPath $strDataPath
!rm $strFileName

--2023-05-08 08:10:00--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘./dataset/cifar-10-python.tar.gz’

100%[======================================>] 170,498,071 52.2MB/s   in 3.1s   

2023-05-08 08:10:04 (52.2 MB/s) - ‘./dataset/cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1
mv: cannot move ‘./dataset/cifar-10-batches-py’ to ‘./dataset/data/cifar-10-batches-py’: Directory not empty


### 2.2 upload to S3 and regiter path to parameter store

In [47]:
strBucketName = sagemaker.Session().default_bucket()
pm.put_params(key="-".join([strPrefix, "BUCKET"]), value=strBucketName, overwrite=True)

'Store suceess'

In [48]:
## upload to S3 and regiter path to parameter store
source_dir, target_bucket, target_dir = os.path.join(strTmpDir, "data"), strBucketName, "-".join([strPrefix, "DATA"])
s3.upload_dir(source_dir, target_bucket, target_dir)
pm.put_params(key="-".join([strPrefix, "DATA-PATH"]), value=f"s3://{target_bucket}/{target_dir}", overwrite=True)

Upload:[./dataset/data] was uploaded to [s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD-DATA]successfully


'Store suceess'

### 2.3 Copy utils to pipeline/utils

In [52]:
from distutils.dir_util import copy_tree

In [53]:
copy_tree("./utils", "./mlops/pipeline/utils")

['./mlops/pipeline/utils/lambda_func.py',
 './mlops/pipeline/utils/monitoringjob_utils.py',
 './mlops/pipeline/utils/ssm.py',
 './mlops/pipeline/utils/s3.py',
 './mlops/pipeline/utils/ecr.py',
 './mlops/pipeline/utils/iam.py',
 './mlops/pipeline/utils/.ipynb_checkpoints/s3-checkpoint.py',
 './mlops/pipeline/utils/.ipynb_checkpoints/iam-checkpoint.py',
 './mlops/pipeline/utils/.ipynb_checkpoints/lambda_func-checkpoint.py',
 './mlops/pipeline/utils/.ipynb_checkpoints/ssm-checkpoint.py',
 './mlops/pipeline/utils/.ipynb_checkpoints/monitoringjob_utils-checkpoint.py',
 './mlops/pipeline/utils/__pycache__/ssm.cpython-39.pyc',
 './mlops/pipeline/utils/__pycache__/s3.cpython-310.pyc',
 './mlops/pipeline/utils/__pycache__/iam.cpython-310.pyc',
 './mlops/pipeline/utils/__pycache__/ssm.cpython-37.pyc',
 './mlops/pipeline/utils/__pycache__/ssm.cpython-310.pyc',
 './mlops/pipeline/utils/__pycache__/monitoringjob_utils.cpython-310.pyc',
 './mlops/pipeline/utils/__pycache__/lambda_func.cpython-310.py

## 3. Set default parameters

In [49]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="-".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "BUCKET"]), value=sagemaker.Session().default_bucket(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "ACCOUNT-ID"]), value=boto3.client("sts").get_caller_identity().get("Account"), overwrite=True)
pm.put_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"]), value=lambdaRoleArn, overwrite=True)
#pm.put_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"]), value=buildRoleArn, overwrite=True)
#pm.put_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"]), value=pipelineRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"]), value=eventBridgeRoleArn, overwrite=True)

'Store suceess'

In [50]:
print ("REGION: ", pm.get_params(key="-".join([strPrefix, "REGION"])))
print ("BUCKET: ", pm.get_params(key="-".join([strPrefix, "BUCKET"])))
print ("SAGEMAKER-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"])))
print ("ACCOUNT-ID: ", pm.get_params(key="-".join([strPrefix, "ACCOUNT-ID"])))
print ("LAMBDA-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"])))
#print ("CODE-BUILD-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"])))
#print ("CODE-PIPELINE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"])))
print ("CODE-EVENTBRIDGE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"])))
print ("DATA-PATH: ", pm.get_params(key="-".join([strPrefix, "DATA-PATH"])))

REGION:  us-east-1
BUCKET:  sagemaker-us-east-1-419974056037
SAGEMAKER-ROLE-ARN:  arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
ACCOUNT-ID:  419974056037
LAMBDA-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-IMD-LabmdaRole
CODE-EVENTBRIDGE-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-IMD-EventBridgeRole
DATA-PATH:  s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD-DATA
